<a href="https://colab.research.google.com/github/ABEERAKHAN-01/Build-a-Simple-Neural-Network-to-Classify-Handwritten-Digits/blob/main/TRAIN_AND_PREDICT_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

def train_and_predict_model(progress=gr.Progress(track_tqdm=True)):
    try:
        # Load and preprocess MNIST dataset
        progress(0, desc="Loading MNIST dataset...")
        (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
        x_train = x_train.astype("float32") / 255.0
        x_test = x_test.astype("float32") / 255.0
        x_train_flat = x_train.reshape(x_train.shape[0], 28 * 28)
        x_test_flat = x_test.reshape(x_test.shape[0], 28 * 28)

        # Build and compile the model
        progress(0.2, desc="Building and compiling model...")
        model = keras.Sequential([
            layers.Dense(256, activation="relu", name="layer1", input_shape=(784,)),
            layers.Dense(128, activation="relu", name="layer2"),
            layers.Dense(10, activation="softmax", name="layer3"),
        ])
        model.compile(
            loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.Adam(1e-3),
            metrics=["accuracy"],
        )

        # Train the model
        progress(0.4, desc="Training model...")
        model.fit(
            x_train_flat, y_train,
            validation_split=0.1,
            epochs=5,
            batch_size=128,
            verbose=0  # Set verbose to 0 to avoid console output flooding
        )
        #saving model

        # This is the line to change
        model.save("mnist_dense.keras")

        # Generate predictions and prepare images for gallery
        progress(0.8, desc="Generating predictions...")
        num_examples = 10
        indices = np.random.choice(len(x_test), size=num_examples, replace=False)
        pred_probs = model.predict(x_test_flat[indices], verbose=0)
        pred_labels = np.argmax(pred_probs, axis=1)

        images_with_labels = []
        for i, idx in enumerate(indices):
            image = x_test[idx]  # This is a numpy array, which is correct for Gradio
            predicted_label = pred_labels[i]
            true_label = y_test[idx]
            label = f"Pred: {predicted_label} | True: {true_label}"
            images_with_labels.append((image, label))

        # Return the final status and the image gallery content
        return "Training and prediction complete.", images_with_labels

    except Exception as e:
        # If an error occurs, return the error message
        return f"An error occurred: {e}", None

# Gradio Blocks interface
with gr.Blocks(title="MNIST Training & Prediction") as demo:
    gr.Markdown("# Model Training and Prediction")
    gr.Markdown("Click the button to train a simple neural network and see its predictions on the MNIST dataset.")

    run_button = gr.Button("Train and Predict")

    status_text = gr.Textbox(label="Status")
    output_gallery = gr.Gallery(label="Predictions", preview=True, columns=5, rows=2, object_fit="contain", height="auto")

    # The click method now has two outputs
    run_button.click(
        fn=train_and_predict_model,
        inputs=None,
        outputs=[status_text, output_gallery]
    )
demo.launch(share=True)